In [1]:
#LBP to Green Channel of Images
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern

# LBP parameters
radius = 2
n_points = 8 * radius
METHOD = 'uniform'

train_dir =r'F:\for Drishti dataset\DRISHTI-gs_Nayab\Training-20211018T055246Z-001\Training\Images'
test_dir = r'F:\for Drishti dataset\DRISHTI-gs_Nayab\Test-20211018T060000Z-001\Test\Images'



train_features = np.empty((0, n_points + 2))
train_labels = []
test_features = np.empty((0, n_points + 2))
test_labels = []

for root, dirs, files in os.walk(train_dir):
    for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
            img = cv2.imread(os.path.join(root, filename))
            #green_channel = img[:, :, 1]  # extract green channel
            #gray = cv2.resize(green_channel, (512, 512), interpolation=cv2.INTER_LINEAR)
            b, g, r = cv2.split(img)
            gray = g
            img = cv2.resize(gray, (512, 512),interpolation = cv2.INTER_LINEAR)
 
            # Calculate LBP image
            lbp = local_binary_pattern(img, n_points, radius, METHOD)

            # Calculate histogram of LBP image
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

            # Normalize histogram
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)

            # Append histogram to array
            train_features = np.vstack((train_features, hist))
            X_label = os.path.basename(root)
            train_labels.append(X_label)


for root, dirs, files in os.walk(test_dir):
    for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
            img = cv2.imread(os.path.join(root, filename))
           # green_channel = img[:, :, 1]  # extract green channel
           # gray = cv2.resize(green_channel, (512, 512), interpolation=cv2.INTER_LINEAR)
            b, g, r = cv2.split(img)
            gray = g
            img = cv2.resize(gray, (512, 512),interpolation = cv2.INTER_LINEAR)
 
            # Calculate LBP image
            lbp = local_binary_pattern(img, n_points, radius, METHOD)

            # Calculate histogram of LBP image
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

            # Normalize histogram
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)

            
            test_features = np.vstack((test_features, hist))
            Y_label = os.path.basename(root)
            test_labels.append(Y_label)
            



np.savetxt('train_features.csv', train_features, delimiter=',')
np.savetxt('train_labels.csv', train_labels, delimiter=',', fmt='%s')
np.savetxt('test_features.csv', test_features, delimiter=',')
np.savetxt('test_labels.csv', test_labels, delimiter=',', fmt='%s')



print('Train Features Shape:', train_features.shape)
print('Train Labels Length:', len(train_labels))
print('Test Features Shape:', test_features.shape)
print('Test Labels Length:', len(test_labels))


Train Features Shape: (50, 18)
Train Labels Length: 50
Test Features Shape: (50, 18)
Test Labels Length: 50


In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score
train_features = np.loadtxt('train_features.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',', dtype=str)
test_features = np.loadtxt('test_features.csv', delimiter=',')
test_labels = np.loadtxt('test_labels.csv', delimiter=',', dtype=str)
#classifier= KNeighborsClassifier(n_neighbors=1, metric='minkowski', p=2 ) Accuracy: 46.0 
#classifier= KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=2 )  #Accuracy: 54.0
#classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 ) #Accuracy: 54.0# 

#classifier= KNeighborsClassifier(n_neighbors=29, metric='minkowski', p=2 )  # Accuracy 70.0
classifier= KNeighborsClassifier(n_neighbors=35, metric='minkowski', p=2 )  

classifier.fit(train_features, train_labels)  

y_pred = classifier.predict(test_features)

# Evaluate the accuracy of the SVM classifier
accuracy = accuracy_score(test_labels, y_pred)

# Print the accuracy of the SVM classifier
print("Accuracy:", accuracy*100)

Accuracy: 74.0


In [3]:
#Decision tree
import numpy as np
from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import accuracy_score
train_features = np.loadtxt('train_features.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',', dtype=str)
test_features = np.loadtxt('test_features.csv', delimiter=',')
test_labels = np.loadtxt('test_labels.csv', delimiter=',', dtype=str)

classifier= DecisionTreeClassifier(criterion='entropy', random_state=42,max_depth=0.25)  


classifier.fit(train_features,train_labels)
y_pred=classifier.predict(test_features)
accuracy=accuracy_score(test_labels,y_pred)
print("Accuracy:", accuracy*100)

Accuracy: 74.0


In [4]:
#Random FOREST
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
train_features = np.loadtxt('train_features.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',', dtype=str)
test_features = np.loadtxt('test_features.csv', delimiter=',')
test_labels = np.loadtxt('test_labels.csv', delimiter=',', dtype=str)

classifier= RandomForestClassifier(n_estimators=14, criterion="entropy",random_state=5)  
classifier.fit(train_features, train_labels)
y_pred=classifier.predict(test_features)
accuracy=accuracy_score(test_labels,y_pred)
print("Accuracy:", accuracy*100)  

Accuracy: 72.0


In [5]:
#Bagging
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
train_features = np.loadtxt('train_features.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',', dtype=str)
test_features = np.loadtxt('test_features.csv', delimiter=',')
test_labels = np.loadtxt('test_labels.csv', delimiter=',', dtype=str)
classifier= BaggingClassifier(n_estimators=10,  random_state=5)  
classifier.fit(train_features, train_labels)
y_pred=classifier.predict(test_features)
accuracy=accuracy_score(test_labels,y_pred)
print("Accuracy:", accuracy*100)  

Accuracy: 66.0


In [6]:
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score

import matplotlib.pyplot as plt
# Load train and test feature matrices and label lists from CSV files
train_features = np.loadtxt('train_features.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',', dtype=str)
test_features = np.loadtxt('test_features.csv', delimiter=',')
test_labels = np.loadtxt('test_labels.csv', delimiter=',', dtype=str)

# Create an SVM classifier with a linear kernel
#clf = svm.SVC(kernel='linear')
clf=svm.SVC(kernel='rbf', C=30, gamma=50) # best parameters for drishti
#clf = svm.SVC(kernel='poly', C=10, gamma=100)
# Train the SVM classifier on the training set
clf.fit(train_features, train_labels)

# Predict the labels of the test set using the trained SVM classifier
y_pred = clf.predict(test_features)

# Evaluate the accuracy of the SVM classifier
accuracy = accuracy_score(test_labels, y_pred)

# Print the accuracy of the SVM classifier
print("Accuracy:", accuracy*100)

confusion_mat = confusion_matrix(test_labels,y_pred)
print("Confusion Matrix")
print(confusion_mat)
f1 = f1_score(test_labels, y_pred, average='weighted')

# Print the F1 score
print("F1 Score:", f1)




Accuracy: 74.0
Confusion Matrix
[[35  2]
 [11  2]]
F1 Score: 0.685272856130404


In [8]:
#merging
merge_dir =r'F:\for Drishti dataset\Merged'
# Set LBP parameters
radius = 3
n_points = 2 * radius
method = 'uniform'
n_bins = n_points + 2

merge_features = np.empty((0, n_points + 2))
merge_labels = []

# Loop through train images and extract LBP features and labels
for root, dirs, files in os.walk(merge_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
             # Read image and extract green channel
            img = cv2.imread(os.path.join(root, file))
            b, g, r = cv2.split(img)
            gray = g
                      
            # Compute LBP features and histogram
            lbp = local_binary_pattern(gray, n_points, radius, method)
            hist, _ = np.histogram(lbp, bins=np.arange(n_bins+1), density=True)
            # Append features and label to train set
            merge_features = np.vstack((merge_features, hist))
            X_label = os.path.basename(root)
            merge_labels.append(X_label)

# Save feature and label matrices into CSV files
np.savetxt('merge_features.csv', merge_features, delimiter=',')
np.savetxt('merge_labels.csv', merge_labels, delimiter=',', fmt='%s')
mean = np.mean(merge_features, axis=0)
std = np.std(merge_features, axis=0)
print('Train Features Mean:', mean)
print('Train Features Standard Deviation:', std)
print('Merge Features Shape:', merge_features.shape)
print('merge Labels Length:', len(merge_labels))

Train Features Mean: [0.12704089 0.12195577 0.06651076 0.07476941 0.07279762 0.1267124
 0.19846362 0.21174953]
Train Features Standard Deviation: [0.00774933 0.00292954 0.00845762 0.01734573 0.01039835 0.00432543
 0.01622077 0.01196198]
Merge Features Shape: (100, 8)
merge Labels Length: 100


In [9]:
merge_features = np.loadtxt('merge_features.csv', delimiter=',')
merge_labels = np.loadtxt('merge_features.csv', delimiter=',', dtype=str)
merge_labels = np.where(merge_labels == 'glaucoma', 0, 1)
print('merge_labels', merge_labels)

merge_labels [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]

In [15]:
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.preprocessing import LabelEncoder
import time


# Create a classifier

clf= DecisionTreeClassifier(criterion='entropy', random_state=42,max_depth=0.25) 
#clf= BaggingClassifier(n_estimators=10,  random_state=5)
#clf = svm.SVC(kernel='linear', C=0.9, gamma=0.5, probability=True)#0.69
#clf=KNeighborsClassifier(n_neighbors=27, metric='minkowski', p=2 ) #0.62

#clf= RandomForestClassifier(n_estimators=14, criterion="entropy",random_state=5)  

# Create a KFold object
k_folds = KFold(n_splits=10)

# List to store cross-validation scores
scores = []
testing_times = []
# Perform cross-validation
for train_index, test_index in k_folds.split(merge_features):
    # Split the data into training and testing sets
    X_train, X_test = merge_features[train_index], merge_features[test_index]
    y_train, y_test = merge_labels[train_index], merge_labels[test_index]

    # Train the classifier
    clf.fit(X_train, y_train)
# Predict labels for test instances and measure testing time
    start_time = time.time()
    y_pred = clf.predict(X_test)
    end_time = time.time()
    testing_time = end_time - start_time
    testing_times.append(testing_time)
    # Evaluate the classifier
    score = clf.score(X_test, y_test)
    scores.append(score)

# Print the cross-validation scores
print("Cross Validation Scores:", scores)
print("Average CV Score:", np.mean(scores))
print("Number of CV Scores used in Average:", len(scores))
# Print the testing times
print("Testing Times:", testing_times)

Cross Validation Scores: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Average CV Score: 1.0
Number of CV Scores used in Average: 10
Testing Times: [0.0, 0.008000612258911133, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
#  Normalize train and test feature matrices
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(merge_features)
np.random.seed(42)
print(merge_labels)

In [7]:
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
import numpy as np
from sklearn import svm
#from sklearn.isotonic import IsotonicRegression
# Define LBP parameters
radius = 2
n_points = 8 * radius
METHOD = 'uniform'
train_csv_file = 'train_features.csv'
train_label_csv = 'train_labels.csv'
test_features = np.empty((0, n_points + 2))
# Load train and test feature matrices and label lists from CSV files
train_features = np.loadtxt(train_csv_file, delimiter=',')
train_labels = np.loadtxt(train_label_csv, delimiter=',', dtype=str)

# Create an SVM classifier with a linear kernel

clf=svm.SVC(kernel='rbf', C=30, gamma=50) # best parameters for drishti
#clf = svm.SVC(kernel='poly', C=131.1, gamma=150)
# Train the SVM classifier on the training set
clf.fit(train_features, train_labels)

url=input("enter url")
img=cv2.imread(url)
b, g, r = cv2.split(img)
gray = g
img = cv2.resize(gray, (512, 512),interpolation = cv2.INTER_LINEAR)

# Calculate LBP image
lbp = local_binary_pattern(img, n_points, radius, METHOD)
# Calculate histogram of LBP image
hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
# Normalize histogram
hist = hist.astype("float")
hist /= (hist.sum() + 1e-7)
 # Append histogram to array
test_features = np.vstack((test_features, hist))
# predict label of test image using trained SVM model
predicted_label = clf.predict(test_features)
print('Predicted label:', predicted_label)

Categories =  np.unique(train_labels)

print(f'Is this right?  (y/n)')
while(True):
  b=input()
  if(b=="y" or b=="n"):
    break
  print("please enter either y or n")

if(b=='n'):
  print("What is the image?")
  for i in range(len(Categories)):
    print(f"Enter {i} for {Categories[i]}")
  k=int(input())
  while(k<0 or k>=len(Categories)):
    print(f"Please enter a valid number between 0-{len(Categories)-1}")
    k=int(input())
  print("Please wait for a while for the model to learn from this image :)")

  # Print the shape of the feature matrix and length of label list for train and test sets
  print('Train Features Shape:', train_features.shape)
  print('Train Labels Length:', len(train_labels))
  train_features = np.vstack((train_features, test_features))
  train_labels = np.append(train_labels,Categories[k])
  np.savetxt(train_csv_file, train_features, delimiter=',')
  np.savetxt(train_label_csv, train_labels, delimiter=',', fmt='%s')
   # Print the shape of the feature matrix and length of label list for train and test sets
  print('Train Features Shape-2:', train_features.shape)
  print('Train Labels Length-2:', len(train_labels))
  
  # Train the SVM classifier on the training set
  clf.fit(train_features, train_labels)

  # predict label of test image using trained SVM model
  predicted_label = clf.predict(test_features)
  print('Predicted label:', predicted_label)

Predicted label: ['glaucoma']
Is this right?  (y/n)
